In [1]:
def ext_scalars(k, n):
    q = k.order()
    assert gcd(n, q) == 1, "Only multiplicative H90, please!"
    
    phi = cyclotomic_polynomial(n, 'z').change_ring(k)
    
    K = k.extension(n, 'x')
    A = K['z']
    h = A(phi.factor()[0][0])
    
    return K, A.quo(h, 'z')

In [2]:
def frob_l(a, i=1):
    A = a.parent()
    q = a.base_ring().base_ring().order()
    return A(list(c^(q^i) for c in a))

In [3]:
def frob_r(a, i=1):
    A = a.parent()
    z = A.gen()
    q = a.base_ring().base_ring().order()
    return sum(c*z^(j*q^i) for j, c in enumerate(a))

In [4]:
def is_h90(a, zeta):
    return frob_l(a) == zeta * a

In [5]:
def solve_h90(A):
    x = A(1)
    n = A.base_ring().degree()
    z = A.gen()
    while not is_h90(x, z):
        x = A.random_element()
        x = sum(frob_l(x, i)*z^-i for i in range(n))
    return x

In [6]:
def cyclo_deg(k, n):
    return Zmod(n)(k.characteristic()).multiplicative_order()

def rnorm(a, m):
    A = a.parent()
    n = A.base_ring().degree()
    r = A.degree()
    q = A.base_ring().base_ring().order()
    s = cyclo_deg(GF(q), m)
    d = n // m
    
#    print n, m, r, s, d
    return product(frob_r(a, s*j) for j in range(d))
def toto(k, m, n):
    q = k.order()
    r = cyclo_deg(k, m)
    s = cyclo_deg(k, n)
    t = (q^s-1)//(q^r-1)
    return t, gcd(t, n), n//m

def change_basis(elem, basis, d2):
    
    k = elem.base_ring()
    d1 = elem.parent().degree()
    A = MatrixSpace(k, d1, d2)()
    
    for i in range(d2):
        L = (basis^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S2 = MatrixSpace(k, d1, 1)
    B = S2(elem.list())
    X = A.solve_right(B)
    
    return X[0,0]

def testl(p, l, n):
    k = GF(p)
    k1, A1 = ext_scalars(k, l^n)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(l^(n+1))
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(l^(n+1)))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = rnorm(h2, l^n)
    aa = h1^(l^n)
    bb = i2^(l^n)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(l^(n+1))
    return h1, A2(c.polynomial().list())*h2

def is_iso(h1, h2):
    m = h1.base_ring().degree()
    n = h2.base_ring().degree()
    l = n // m
    i2 = rnorm(h2, m)
    j2 = change_basis(i2, h2.parent().gen()^l, h1.parent().degree())
    return h1.list()[0].minpoly() == j2.minpoly()  
    
def frob_rc(a, n, m, i=1):
    A = a.parent()
    z = A.gen()
    l = n // m
    u, v = xgcd(l, m)[1:]
    q = a.base_ring().base_ring().order()
    return sum(c*z^(u*l*j)*z^(v*m*j*q^i) for j, c in enumerate(a))

def rnormc(a, m):
    A = a.parent()
    n = a.base_ring().degree()
    q = A.base_ring().base_ring().order()
    l = n // m
    d = cyclo_deg(GF(q), l)
    print d, l
    return product(frob_rc(a, n, m, j) for j in range(d))

"""
Here we find an element h2 solution of H90 in F_p^n compatible with h1.
"""
def findh90(h1, n):
    k = h1.base_ring().base_ring()
    k1, A1 = h1.base_ring(), h1.parent()
    M1 = A1.modulus()
    R = k['z']
    m = k1.degree()
    l = n // m
    k2 = k.extension(n)
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(M1.change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h2 = solve_h90(A2)
    i2 = rnorm(h2, m)
    aa = h1^(m)
    bb = i2^(m)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(n)
    return A2(c.polynomial().list())*h2

"""
Return two elements h1, h2 that are respectively solutions of H90 in 
A_m = F_p^m ⊗ F_p(ζ_m) for the root 1 ⊗ ζ_m and in A_n for 1 ⊗ ζ_n, such that
N(h2) = h1.
"""
def testc(p, m, n):
    k = GF(p)
    k1, A1 = ext_scalars(k, m)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(n)
    R2 = k2['z']
    x = R.gen()
    l = n // m
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = rnorm(h2, m)
    aa = h1^m
    bb = i2^m
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(n)
    return h1, A2(c.polynomial().list())*h2

In [9]:
K, A = ext_scalars(GF(7), 9)  # 1 = r₁ < r₂ = 2
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h)*frob_r(h, 2)
v = rnorm(h, 3)
is_h90(u, A.gen()^3), frob_l(u, 3) == u, u == v
A.degree()

3

In [29]:
K, A = ext_scalars(GF(19), 9)  # 1 = r₁ = r₂
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h)*frob_r(h, 2)
v = rnorm(h, 3)
is_h90(u, A.gen()^3), frob_l(u, 3) == u, u == v

9 3 1 1 1


(True, True, False)

In [41]:
K, A = ext_scalars(GF(7), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)
v = rnorm(h, 3)
is_h90(u, A.gen()^10), frob_r(u, 3) == u, u == v

15 3 4 1 4


(True, True, True)

In [46]:
K, A = ext_scalars(GF(7), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 4)*frob_r(h, 8)
v = rnorm(h, 5)
is_h90(u, A.gen()^3), frob_r(u, 4) == u, is_h90(v, A.gen()^3), frob_r(v, 4) == v, u == v, u == h^3

15 5 4 4 1


(True, True, False, True, False, True)

In [100]:
K, A = ext_scalars(GF(19), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 1)
is_h90(u, A.gen()^5), frob_l(u, 3) == u

(False, False)

# An $\ell$-adic tower

In [94]:
p, l, n = 17, 3, 2
cyclo_deg(GF(p), l^n)

2

In [95]:
for i in range(10):
    h1, h2 = testl(p, l, n)
    print is_iso(h1, h2)

True
True
True
True
True
True
True
True
True
True


In [16]:
h2 == h2b

False

In [17]:
is_iso(h1, h2b)

True

In [18]:
h3 = findh90(h2b, 3^3)

In [19]:
is_iso(h1, h3)

True

# Composita

In [113]:
p = 2
k = GF(p)
cyclo_deg(k, 3^2*5)

12

In [114]:
m = 3^1 * 5
n = 3^2 * 5
l = n // m
for i in range(10):
    h1, h2 = testc(p, m, n)
    print is_iso(h1, h2)

True
True
True
True
True
True
True
True
True
True


In [115]:
h1

(x^14 + x^10 + x^7 + x^4 + x^3 + x^2 + x)*z^3 + (x^14 + x^13 + x^10 + x^9 + x^7 + x^5 + x^4 + x^3 + x^2 + 1)*z^2 + (x^14 + x^11 + x^9 + x^8 + x^6 + x^3 + x^2 + x + 1)*z + x^14 + x^13 + x^11 + x^10 + x^9 + x^7 + x^6

In [116]:
h2

(z45^43 + z45^42 + z45^40 + z45^39 + z45^37 + z45^34 + z45^32 + z45^31 + z45^29 + z45^27 + z45^26 + z45^25 + z45^24 + z45^22 + z45^18 + z45^15 + z45^14 + z45^12 + z45^11 + 1)*z^11 + (z45^44 + z45^41 + z45^39 + z45^38 + z45^36 + z45^35 + z45^33 + z45^32 + z45^30 + z45^28 + z45^25 + z45^22 + z45^20 + z45^17 + z45^14 + z45^12 + z45^7 + z45^6 + z45^2 + 1)*z^10 + (z45^44 + z45^43 + z45^42 + z45^38 + z45^37 + z45^36 + z45^29 + z45^27 + z45^26 + z45^24 + z45^23 + z45^20 + z45^17 + z45^15 + z45^13 + z45^12 + z45^10 + z45^9 + z45^8 + z45^6 + z45^4 + z45^3 + z45)*z^9 + (z45^43 + z45^41 + z45^37 + z45^36 + z45^35 + z45^32 + z45^31 + z45^30 + z45^28 + z45^27 + z45^26 + z45^25 + z45^24 + z45^21 + z45^17 + z45^15 + z45^14 + z45^12 + z45^11 + z45^9 + z45^8 + z45^7 + z45^6 + z45^5 + z45^2 + z45)*z^8 + (z45^42 + z45^41 + z45^39 + z45^38 + z45^37 + z45^36 + z45^34 + z45^32 + z45^28 + z45^27 + z45^26 + z45^24 + z45^21 + z45^19 + z45^18 + z45^16 + z45^15 + z45^14 + z45^13 + z45^12 + z45^8 + z45^5 + z45^4 

In [7]:
for x in rnorm(h2, m).minpoly().list():
    print x.minpoly()

NameError: name 'h2' is not defined

# A compatibility test using norms

We test if the usage of norms (in the case where it should work) is indeed compatible.

In [7]:
def basis_matrix(a, n = None):
    K = a.parent()
    m = K.degree()
    
    if n == None:
        n = m
    
    k = K.prime_subfield()
    S = MatrixSpace(k, m, n)()
    for j in range(n):
        L = (a^j).polynomial().list()
        i = 0
        for l in L:
            S[i, j] = l
            i += 1
    return S

def fflist(x):
    L = x.polynomial().list()
    l = len(L)
    k = x.parent()
    d = k.degree()
    if l < d:
        L += (d-l)*[k()]
        
    return L

def compute_map(a, b):
    
    A = basis_matrix(a)
    B = basis_matrix(b, a.parent().degree())
    C = B*A^(-1)
    
    K = b.parent()
    k = K.prime_subfield()
    S = MatrixSpace(k, a.parent().degree(), 1)

    return lambda x : K((C*S(fflist(x))).column(0))

def compute_emb(h1, h2):
    m = h1.base_ring().degree()
    n = h2.base_ring().degree()
    l = n // m
    i2 = rnorm(h2, m)
    j2 = change_basis(i2, h2.parent().gen()^l, h1.parent().degree())
    return compute_map(h1.list()[0], j2) 

In [170]:
p = 23
m, n, o = 15, 45, 135
k = GF(p)
km, Am = ext_scalars(k, m)
hm = solve_h90(Am)
hn = findh90(hm, n)
ho = findh90(hn, o)

f = compute_emb(hm, hn)
g = compute_emb(hn, ho)
h = compute_emb(hm, ho)

g(f(km.gen())) == h(km.gen())

True

In [177]:
for x in rnorm(hn, m).list():
    print(x.minpoly())

x^15 + 18*x^11 + 3*x^10 + 8*x^9 + 16*x^8 + 13*x^6 + 3*x^4 + 17*x^3 + 11*x^2 + 5*x + 5
x
x
x^15 + 22*x^11 + 17*x^10 + 8*x^9 + 20*x^8 + 5*x^7 + 13*x^6 + 9*x^5 + 20*x^4 + 19*x^3 + 8*x^2 + 21*x + 10
x
x
x^15 + 7*x^11 + 5*x^10 + 3*x^9 + 20*x^8 + 7*x^7 + 12*x^6 + 15*x^5 + 20*x^4 + 16*x^3 + 21*x^2 + 5*x + 4
x
x
x^15 + 18*x^11 + 20*x^10 + 8*x^9 + 7*x^8 + 10*x^6 + 20*x^4 + 17*x^3 + 12*x^2 + 5*x + 18
x
x


In [182]:
for x in rnorm(hn, m).minpoly():
    print x.minpoly()

x + 1
x^15 + 21*x^11 + 2*x^10 + 6*x^9 + 6*x^8 + 19*x^7 + 18*x^6 + 3*x^5 + 13*x^4 + 20*x^3 + 17*x^2 + 4*x + 15
x^15 + 22*x^13 + 4*x^12 + 5*x^11 + 3*x^10 + 17*x^9 + 9*x^8 + 20*x^7 + 11*x^6 + 6*x^5 + x^4 + 9*x^3 + 20*x^2 + 9*x + 21
x^15 + 21*x^11 + 8*x^10 + 17*x^9 + 2*x^8 + 5*x^7 + 14*x^6 + 8*x^5 + 16*x^4 + 2*x^3 + 6*x^2 + 20*x + 17
x + 22


In [183]:
for x in hm.minpoly():
    print x.minpoly()

x + 1
x^15 + 21*x^11 + 2*x^10 + 6*x^9 + 6*x^8 + 19*x^7 + 18*x^6 + 3*x^5 + 13*x^4 + 20*x^3 + 17*x^2 + 4*x + 15
x^15 + 22*x^13 + 4*x^12 + 5*x^11 + 3*x^10 + 17*x^9 + 9*x^8 + 20*x^7 + 11*x^6 + 6*x^5 + x^4 + 9*x^3 + 20*x^2 + 9*x + 21
x^15 + 21*x^11 + 8*x^10 + 17*x^9 + 2*x^8 + 5*x^7 + 14*x^6 + 8*x^5 + 16*x^4 + 2*x^3 + 6*x^2 + 20*x + 17
x + 22


# A compatibility test using CRT

In [40]:
"""
Test function to work around the extension F_{p^l}, F_{p^{lm}}, F_{p^{l²m²}}.
"""
def test_comp(p, l, m):
    
    # We create the prime field, the root ζ_{l²m²}, and the field k(ζ_{l²m²})
    
    k = GF(p)
    R = k['z']
    Cyclo = cyclotomic_polynomial(l^2*m^2)
    Zl2m2 = Cyclo.change_ring(k).factor()[0][0]
    Cl2m2.<z> = k.extension(Zl2m2)
    
    # We create the other roots using ζ_{l²m²} (we are cheating a bit...)
    
    zl2 = z^(m^2)
    zm2 = z^(l^2)
    Zl2 = zl2.minpoly()
    Zm2 = zm2.minpoly()
    
    zl = zl2^l
    zm = zm2^m
    Zl = zl.minpoly()
    Zm = zm.minpoly()
    
    u1, u2 = xgcd(l, m)[1:3]
    zlm = zl^u2 * zm^u1
    Zlm = zlm.minpoly()
    
    # We create the rings and algebras
    
    kl2m2 = k.extension(l^2*m^2, "x")
    Rl2m2 = kl2m2["T"]
    Al2m2 = Rl2m2.quo(Zl2m2)
    
    kl2 = k.extension(l^2, "x")
    Rl2 = kl2["T"]
    Al2 = Rl2.quo(Zl2)

    km2 = k.extension(m^2, "x")
    Rm2 = km2["T"]
    Am2 = Rm2.quo(Zm2)
    
    klm = k.extension(l*m, "x")
    Rlm = klm["T"]
    Alm = Rlm.quo(Zlm)
    
    kl = k.extension(l, "x")
    Rl = kl["T"]
    Al = Rl.quo(Zl)
    
    km = k.extension(m, "x")
    Rm = km["T"]
    Am = Rm.quo(Zm)
    
    # We compute the solutions of the Hilbert 90 problems in the first extension of each tower
    
    hl = solve_h90(Al)
    hm = solve_h90(Am)
    
    # We compute compatible solutions in higher extensions of the two towers
    ## l-adic tower
    
    htmpl2 = solve_h90(Al2)
    il2 = rnorm(htmpl2, l)
    Cl2 = k.extension(Zl2, 'x')
    aal = hl^(l)
    bbl = il2^(l)
    al = R([x for x in aal.list()]).subs(Cl2.gen()^l)
    bl = Cl2([k(x) for x in bbl.list()])
    
    tl = al/bl
    cl = tl.nth_root(l^2)
    hl2 = Al2(cl.polynomial().list())*htmpl2
    
    ## m-adic tower
    
    htmpm2 = solve_h90(Am2)
    im2 = rnorm(htmpm2, m)
    Cm2 = k.extension(Zm2, 'x')
    aam = hm^(m)
    bbm = im2^(m)
    am = R([x for x in aam.list()]).subs(Cm2.gen()^m)
    bm = Cm2([k(x) for x in bbm.list()])
    
    tm = am/bm
    cm = tm.nth_root(m^2)
    hm2 = Am2(cm.polynomial().list())*htmpm2
    
    # We compute compatible solutions in the composita
    ## l × m
    
    Clm = k.extension(Zlm, 'x')
    htmplm = solve_h90(Alm)
    alm = Clm([k(x) for x in (htmplm^(l*m)).list()])
    blm = Clm([k(x) for x in (hl^l).list()]).polynomial().subs(Clm.gen()^m)
    clm = Clm([k(x) for x in (hm^m).list()]).polynomial().subs(Clm.gen()^l)
    
    tl_lm = (blm*alm^(-1)).nth_root(l)
    tm_lm = (clm*alm^(-1)).nth_root(m)
    
    hlm = Alm(tl_lm.polynomial().list())*htmplm^(m*u2)+Alm(tm_lm.polynomial().list())*htmplm^(l*u1)
    
    Tlm_m = lambda x : Alm((clm/blm).polynomial().list())*x^l
    Tlm_l = lambda x : Alm((blm/clm).polynomial().list())*x^m
    
    return hlm, hl, Tlm_l

    ## l² × m²
    
    v1, v2 = xgcd(l^2, m^2)[1:3]
    Cl2m2 = k.extension(Zl2m2, 'x')
    htmpl2m2 = solve_h90(Al2m2)
    al2m2 = Cl2m2([k(x) for x in (htmpl2m2^(l^2*m^2)).list()])
    bl2m2 = Cl2m2([k(x) for x in (hl2^(l^2)).list()]).polynomial().subs(Cl2m2.gen()^(m^2))
    cl2m2 = Cl2m2([k(x) for x in (hm2^(m^2)).list()]).polynomial().subs(Cl2m2.gen()^(l^2))
    
    tl2_l2m2 = (bl2m2*al2m2^(-1)).nth_root(l^2)
    tm2_l2m2 = (cl2m2*al2m2^(-1)).nth_root(m^2)
    
    hl2m2 = Al2m2(tl2_l2m2.polynomial().list())*htmpl2m2^(m^2*v1)+Al2m2(tm2_l2m2.polynomial().list())*htmpl2m2^(l^2*v2)
    
    return hl2m2

In [11]:
p = 23
k = GF(p)
l, m = 3, 5

In [101]:
P.change_ring(k).factor()[0]

(x^60 + 6*x^45 + 21*x^30 + 16*x^15 + 1, 1)

In [43]:
%time hlm, hl, Tlm_l = test_comp(p, l, m)

CPU times: user 11.8 s, sys: 28 ms, total: 11.9 s
Wall time: 11.8 s


In [44]:
is_h90(hlm, parent(hlm).gen())

False

Too bad, it should be true.